In [1]:
import os
os.chdir("datasetfm")
Dataset='Dataset'
Data_Dir=os.listdir(Dataset)
print(Data_Dir)
print('Total no of images:',len(os.listdir('Dataset/with_mask')) + len(os.listdir('Dataset/without_mask')))
print('No of images with mask:',len(os.listdir('Dataset/with_mask')))
print('No of images without mask:',len(os.listdir('Dataset/without_mask')))

['without_mask', 'with_mask']
Total no of images: 781
No of images with mask: 396
No of images without mask: 385


In [2]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
img_rows, img_cols = 112, 112

images = []
labels = []

for category in Data_Dir:
    folder_path = os.path.join(Dataset, category)
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        img=cv2.imread(img_path)

        try:
            grayscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized_img=cv2.resize(grayscale_img,(img_rows, img_cols))
            images.append(resized_img)
            labels.append(category)
        except Exception as e:
            print('Exception:',e)
        
images=np.array(images)/255.0
images=np.reshape(images,(images.shape[0],img_rows, img_cols,1))
#ONE HOT ENCODER
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
labels = np.array(labels)

(train_X, test_X, train_y, test_y) = train_test_split(images, labels, test_size=0.25, random_state=0)
print('No of images in training sample:',len(train_X))
print('No of images in testing sample:', len(test_X))

ModuleNotFoundError: No module named 'cv2'

In [3]:

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D


num_classes = 2
batch_size = 32


model=Sequential()


model.add(Conv2D(64,(3,3),input_shape=(img_rows, img_cols, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))


model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

print(model.summary())

ModuleNotFoundError: No module named 'keras'

In [4]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='face_mask_detection_architecture.png')

from keras.optimizers import Adam

epochs = 50

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

fitted_model = model.fit(
    train_X,
    train_y,
    epochs = epochs,
    validation_split=0.25)


from matplotlib import pyplot as plt

plt.plot(fitted_model.history['loss'],'r',label='training loss')
plt.plot(fitted_model.history['val_loss'],label='validation loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss Value')
plt.legend()
plt.show()

plt.plot(fitted_model.history['accuracy'],'r',label='training accuracy')
plt.plot(fitted_model.history['val_accuracy'],label='validation accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy Value')
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'keras'

In [5]:
model.save('facemaskpredictor.h5')

NameError: name 'model' is not defined